# Notebook 5 : Web scraping

In [1]:
# Décommenter la ligne suivante pour installer lxml (nécessaire pour read_html)
#%pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 45.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [220]:
import pandas as pd
import requests

from bs4 import BeautifulSoup

## Bonsaïs

Le site [Umi Zen Bonsai](https://umizenbonsai.com/) est une boutique de vente en ligne dédiée aux bonsaïs. Les conifères sont disponible sur la page web [https://umizenbonsai.com/shop/bonsai/coniferes/](https://umizenbonsai.com/shop/bonsai/coniferes/). Comme beaucoup d'autres sites, l'information est organisée en blocs dans lesquels il est possible de récupérer des données.

Pour scraper ce type de site, le processus consiste à capturer les blocs dans un premier temps, puis à en extraire les données.

1. Récupérer le contenu de la page avec `requests` et passer le résultat au parser de `BeautifulSoup`.

In [221]:
url_bonsai = "https://umizenbonsai.com/shop/bonsai/coniferes/"
r_bonsai = requests.get(url_bonsai)
if r_bonsai.status_code == 200:
    print("Page récupérée")
else:
    print(f"Erreur {r_bonsai.status_code}")
soup_bonsai = BeautifulSoup(r_bonsai.text, "html.parser")
print(soup_bonsai)

Page récupérée
<!DOCTYPE html>

<html class="html" lang="fr-FR">
<head><meta charset="utf-8"/><script>if(navigator.userAgent.match(/MSIE|Internet Explorer/i)||navigator.userAgent.match(/Trident\/7\..*?rv:11/i)){var href=document.location.href;if(!href.match(/[?&]nowprocket/)){if(href.indexOf("?")==-1){if(href.indexOf("#")==-1){document.location.href=href+"?nowprocket=1"}else{document.location.href=href.replace("#","?nowprocket=1#")}}else{if(href.indexOf("#")==-1){document.location.href=href+"&nowprocket=1"}else{document.location.href=href.replace("#","&nowprocket=1#")}}}}</script><script>(()=>{class RocketLazyLoadScripts{constructor(){this.v="2.0.3",this.userEvents=["keydown","keyup","mousedown","mouseup","mousemove","mouseover","mouseenter","mouseout","mouseleave","touchmove","touchstart","touchend","touchcancel","wheel","click","dblclick","input","visibilitychange"],this.attributeEvents=["onblur","onclick","oncontextmenu","ondblclick","onfocus","onmousedown","onmouseenter","onmousele

2. Écrire un sélecteur CSS pour capturer les éléments `li` qui contiennent les blocs correspondants aux bonsaïs. Vérifier sur le site que le nombre de bonsaïs affichés correspond.

In [ ]:
selector_bonsai = "li.entry"
#selector_bonsai = "ul.products > li"
bonsais = soup_bonsai.select(selector_bonsai)
#print(f"{len(bonsais)} bonsais")
bonsais

[<li class="entry has-media col span_1_of_4 owp-content-center owp-thumbs-layout-horizontal owp-btn-big owp-tabs-layout-horizontal circle-sale product type-product post-55126 status-publish first instock product_cat-coniferes product_cat-tous-les-bonsai product_tag-bonsai has-post-thumbnail shipping-taxable product-type-simple">
 <div class="product-inner clr">
 <div class="woo-entry-image clr">
 <a class="woocommerce-LoopProduct-link" href="https://umizenbonsai.com/vente/bonsai-cypres-du-japon/"><img alt="Bonsai Cyprès du Japon – 33cm" class="woo-entry-image-main" decoding="async" fetchpriority="high" height="800" itemprop="image" sizes="(max-width: 800px) 100vw, 800px" src="https://umizenbonsai.com/wp-content/uploads/2024/06/bonsai-cypres-du-japon-umi-zen-bonsai-shop.webp" srcset="https://umizenbonsai.com/wp-content/uploads/2024/06/bonsai-cypres-du-japon-umi-zen-bonsai-shop.webp 800w, https://umizenbonsai.com/wp-content/uploads/2024/06/bonsai-cypres-du-japon-umi-zen-bonsai-shop-150x1

3. Écrire une fonction qui prend un bloc de la liste précédente et retourne un tuple contenant le nom, le prix et le lien de description du bonsaï.

In [166]:
def bonsai_info(node):
    selector_price = "span.price bdi"
    selector_nom = "li.title h2 a"
    node_price = node.select(selector_price)[-1]
    node_nom = node.select(selector_nom)[0]
    return {
    "nom" : node_nom.text,
    "price" : node_price.text,
    "url" : node_nom.attrs["href"]
    }

4. Utiliser les deux questions précédentes pour construire un dataframe contenant les données des bonsaïs.

In [167]:
bonsais_df = pd.DataFrame([bonsai_info(node) for node in bonsais])
bonsais_df

,nom,price,url
0,Bonsai Cyprès du Japon – 33cm,140.00€,https://umizenbonsai.com/vente/bonsai-cypres-d...
1,Bonsai Genévrier de Phénicie – 59cm,225.00€,https://umizenbonsai.com/vente/bonsai-genevrie...
2,Bonsai Genévrier de Phénicie – 67cm,333.00€,https://umizenbonsai.com/vente/bonsai-genevrie...
3,Bonsai If Commun – 58cm,"1,190.00€",https://umizenbonsai.com/vente/bonsai-if-commu...
4,Bonsai If Commun – 85cm,585.00€,https://umizenbonsai.com/vente/bonsai-if-commun/
5,Bonsai If du Japon – 50cm,750.00€,https://umizenbonsai.com/vente/bonsai-if-du-ja...
6,Bonsai If du Japon – 63cm,"1,290.00€",https://umizenbonsai.com/vente/bonsai-if-japon/
7,Bonsai Pin Sylvestre – 38cm,440.00€,https://umizenbonsai.com/vente/bonsai-pin-sylv...
8,Bonsai Pin Sylvestre – 65cm,370.00€,https://umizenbonsai.com/vente/bonsai-pin-sylv...


5. (*Bonus*) Écrire une fonction pour récupérer la provenance, le feuilage et les dimension du bonsaï à partir du lien de description. Utiliser cette fonction pour ajouter des colonnes au dataframe précédent

In [ ]:
def get_infos2(url_current):
    r_current = requests.get(url_current)
    soup_current = BeautifulSoup(r_current.text, "html.parser")
    selector_current = "div.elementor-element div div p"
    provenance = soup_current.select(selector_current)[0].text
    feuillage = soup_current.select(selector_current)[1].text
    dimension = soup_current.select(selector_current)[7].text
    return {
    "provenance" : provenance,
    "feuillage" : feuillage,
    "dimension" : dimension
    }

In [169]:
bonsais_infos2  = pd.DataFrame([get_infos2(url) for url in bonsais_df.url])
bonsais_infos2.columns=["provenance", "feuillage", "dimensions"]
bonsais_infos2

,provenance,feuillage,dimensions
0,Japon,Persistant,Nebari environ 2cm Hauteur hors pot : 33cm
1,Yamadori France,Persistant,Nebari environ 3cm Hauteur hors pot : 59cm
2,Yamadori France,Persistant,Nebari environ 15cm Hauteur hors pot : 67cm
3,Yamadori France,"Persistant. Très toxique, ne pas ingérer",Nebari environ 10cm Hauteur hors pot : 58cm
4,Yamadori France,"Persistant. Très toxique, ne pas ingérer",Nebari environ 10cm Hauteur hors pot : 85cm
5,Japon,"Persistant. Très toxique, ne pas ingérer",Nebari environ 12cm Hauteur hors pot : 50cm
6,Japon,"Persistant. Très toxique, ne pas ingérer",Nebari environ 10cm Hauteur hors pot : 63cm
7,Yamadori France,Persistant,Nebari environ 5cm Hauteur hors pot : 38cm
8,Yamadori France,Persistant,Nebari environ 7cm Hauteur hors pot : 65cm


In [172]:
pd.concat([bonsais_df, bonsais_infos2], axis=1)

,nom,price,url,provenance,feuillage,dimensions
0,Bonsai Cyprès du Japon – 33cm,140.00€,https://umizenbonsai.com/vente/bonsai-cypres-d...,Japon,Persistant,Nebari environ 2cm Hauteur hors pot : 33cm
1,Bonsai Genévrier de Phénicie – 59cm,225.00€,https://umizenbonsai.com/vente/bonsai-genevrie...,Yamadori France,Persistant,Nebari environ 3cm Hauteur hors pot : 59cm
2,Bonsai Genévrier de Phénicie – 67cm,333.00€,https://umizenbonsai.com/vente/bonsai-genevrie...,Yamadori France,Persistant,Nebari environ 15cm Hauteur hors pot : 67cm
3,Bonsai If Commun – 58cm,"1,190.00€",https://umizenbonsai.com/vente/bonsai-if-commu...,Yamadori France,"Persistant. Très toxique, ne pas ingérer",Nebari environ 10cm Hauteur hors pot : 58cm
4,Bonsai If Commun – 85cm,585.00€,https://umizenbonsai.com/vente/bonsai-if-commun/,Yamadori France,"Persistant. Très toxique, ne pas ingérer",Nebari environ 10cm Hauteur hors pot : 85cm
5,Bonsai If du Japon – 50cm,750.00€,https://umizenbonsai.com/vente/bonsai-if-du-ja...,Japon,"Persistant. Très toxique, ne pas ingérer",Nebari environ 12cm Hauteur hors pot : 50cm
6,Bonsai If du Japon – 63cm,"1,290.00€",https://umizenbonsai.com/vente/bonsai-if-japon/,Japon,"Persistant. Très toxique, ne pas ingérer",Nebari environ 10cm Hauteur hors pot : 63cm
7,Bonsai Pin Sylvestre – 38cm,440.00€,https://umizenbonsai.com/vente/bonsai-pin-sylv...,Yamadori France,Persistant,Nebari environ 5cm Hauteur hors pot : 38cm
8,Bonsai Pin Sylvestre – 65cm,370.00€,https://umizenbonsai.com/vente/bonsai-pin-sylv...,Yamadori France,Persistant,Nebari environ 7cm Hauteur hors pot : 65cm


## Trampoline

Le trampoline est un sport olympique depuis les jeux de Sydney en 2000. La page suivante contient les listes des hommes et des femmes ayant obtenu une médaille olympique dans cette discipline :
[https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline](https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline)

Un tableau est contenu dans un élément `table` avec des balises pour les lignes `tr`, pour les colonnes `th`, pour les cellules `td`, ... Cela peut être fastidieux à scraper et très répétitif. Heureusement, Pandas propose la fonction `read_html` pour récupérer des tableaux sous forme de dataframes à partir d'une page web.

1. Utiliser la fonction `read_html` de Pandas sur la page des médaillés olympiques au trampoline. Combien de dataframes sont récupérés ?

In [175]:
url = "https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline"
len(pd.read_html(url))

6

2. Extraire de la liste précédente les dataframes des médailles masculines et féminines.

In [186]:
medailles_hommes = pd.read_html(url)[0]
medailles_femmes = pd.read_html(url)[1]

3. À partir de ces dataframes, compter combien chaque pays a reçu de médailles d'or, d'argent et de bronze.

In [187]:
medailles=pd.concat([medailles_hommes, medailles_femmes], axis=0)
medailles.reset_index(inplace=True)
medailles

,index,Édition,Lieu,Or,Argent,Bronze
0,0,2000,Sydney,Alexander Moskalenko (RUS),Ji Wallace (AUS),Mathieu Turgeon (CAN)
1,1,2004,Athènes,Yuri Nikitin (UKR),Alexander Moskalenko (RUS),Henrik Stehlik (GER)
2,2,2008,Pékin,Lu Chunlong (CHN),Jason Burnett (CAN),Dong Dong (CHN)
3,3,2012,Londres,Dong Dong (CHN),Dmitry Ushakov (RUS),Lu Chunlong (CHN)
4,4,2016,Rio de Janeiro,Uladzislau Hancharou (BLR),Dong Dong (CHN),Gao Lei (CHN)
5,5,2020,Tokyo,Ivan Litvinovich (BLR),Dong Dong (CHN),Dylan Schmidt (NZL)
6,6,2024,Paris,Ivan Litvinovich (AIN),Wang Zisai (CHN),Yan Langyu (CHN)
7,0,2000,Sydney,Irina Karavaeva (RUS),Oksana Tsyhulyeva (UKR),Karen Cockburn (CAN)
8,1,2004,Athènes,Anna Dogonadze (GER),Karen Cockburn (CAN),Huang Shanshan (CHN)
9,2,2008,Pékin,He Wenna (CHN),Karen Cockburn (CAN),Ekaterina Khilko (UZB)


In [190]:
medailles["pays_or"] = medailles.Or.apply(lambda v:v[-4:-1])
medailles["pays_argent"] = medailles.Argent.apply(lambda v:v[-4:-1])
medailles["pays_bronze"] = medailles.Bronze.apply(lambda v:v[-4:-1])
medailles

,index,Édition,Lieu,Or,Argent,Bronze,pays_or,pays_argent,pays_bronze
0,0,2000,Sydney,Alexander Moskalenko (RUS),Ji Wallace (AUS),Mathieu Turgeon (CAN),RUS,AUS,CAN
1,1,2004,Athènes,Yuri Nikitin (UKR),Alexander Moskalenko (RUS),Henrik Stehlik (GER),UKR,RUS,GER
2,2,2008,Pékin,Lu Chunlong (CHN),Jason Burnett (CAN),Dong Dong (CHN),CHN,CAN,CHN
3,3,2012,Londres,Dong Dong (CHN),Dmitry Ushakov (RUS),Lu Chunlong (CHN),CHN,RUS,CHN
4,4,2016,Rio de Janeiro,Uladzislau Hancharou (BLR),Dong Dong (CHN),Gao Lei (CHN),BLR,CHN,CHN
5,5,2020,Tokyo,Ivan Litvinovich (BLR),Dong Dong (CHN),Dylan Schmidt (NZL),BLR,CHN,NZL
6,6,2024,Paris,Ivan Litvinovich (AIN),Wang Zisai (CHN),Yan Langyu (CHN),AIN,CHN,CHN
7,0,2000,Sydney,Irina Karavaeva (RUS),Oksana Tsyhulyeva (UKR),Karen Cockburn (CAN),RUS,UKR,CAN
8,1,2004,Athènes,Anna Dogonadze (GER),Karen Cockburn (CAN),Huang Shanshan (CHN),GER,CAN,CHN
9,2,2008,Pékin,He Wenna (CHN),Karen Cockburn (CAN),Ekaterina Khilko (UZB),CHN,CAN,UZB


In [216]:
liste_pays = pd.concat([medailles.pays_or, medailles.pays_argent, medailles.pays_bronze], axis=0).drop_duplicates().to_frame()
liste_pays.columns = ["pays"]
liste_pays

,pays
0,RUS
1,UKR
2,CHN
4,BLR
6,AIN
8,GER
10,CAN
13,GBR
0,AUS
5,NZL


In [217]:
def nb_medailles(pays):
    return(len(medailles[medailles.pays_or==pays]) + len(medailles[medailles.pays_argent==pays]) + len(medailles[medailles.pays_bronze==pays]))

print(nb_medailles("CHN"))

16


In [218]:
liste_pays["nb_medailles_total"] = liste_pays.pays.apply(nb_medailles)
liste_pays.sort_values(by="nb_medailles_total", ascending=False)

,pays,nb_medailles_total
2,CHN,16
10,CAN,8
0,RUS,4
13,GBR,3
1,UKR,2
6,AIN,2
4,BLR,2
8,GER,2
0,AUS,1
5,NZL,1


4. (*Bonus*) Construire un dataframe contenant, pour chaque pays, le nombre de médailles d'or, d'argent et de bronze ainsi que le nombre total de médailles. Classer ce dataframe dans l'ordre usuel en fonction d'abord du nombre de médailles d'or, puis du nombre de médailles d'argent et enfin du nombre de médailles de bronze. Comparer le résultat avec le tableau des médailles sur la page [https://fr.wikipedia.org/wiki/Trampoline_aux_Jeux_olympiques](https://fr.wikipedia.org/wiki/Trampoline_aux_Jeux_olympiques).

In [219]:
def nb_medailles_or(pays):
    return(len(medailles[medailles.pays_or==pays]))
def nb_medailles_argent(pays):
    return(len(medailles[medailles.pays_argent==pays]))
def nb_medailles_bronze(pays):
    return(len(medailles[medailles.pays_bronze==pays]))

liste_pays["nb_medailles_or"] = liste_pays.pays.apply(nb_medailles_or)
liste_pays["nb_medailles_argent"] = liste_pays.pays.apply(nb_medailles_argent)
liste_pays["nb_medailles_bronze"] = liste_pays.pays.apply(nb_medailles_bronze)
liste_pays.sort_values(by=["nb_medailles_or", "nb_medailles_argent", "nb_medailles_bronze"], ascending=False)

,pays,nb_medailles_total,nb_medailles_or,nb_medailles_argent,nb_medailles_bronze
2,CHN,16,4,5,7
10,CAN,8,2,3,3
0,RUS,4,2,2,0
4,BLR,2,2,0,0
13,GBR,3,1,1,1
1,UKR,2,1,1,0
6,AIN,2,1,1,0
8,GER,2,1,0,1
0,AUS,1,0,1,0
5,NZL,1,0,0,1


## Cate Blanchett

Dans le cours, nous avons essayé de trouver avec quels acteurs Cate Blanchett a joué le plus au cours des années 2000. Pour cela, nous avons récupéré la liste des pages Wikipedia des films où elle tient un rôle avec le code suivant :

In [79]:
url_wikipedia = "https://fr.wikipedia.org"
url_blanchett = url_wikipedia + "/wiki/Cate_Blanchett"

r_blanchett = requests.get(url_blanchett)
assert r_blanchett.status_code == 200, f"Erreur {r_blanchett.status_code}"

soup_blanchett = BeautifulSoup(r_blanchett.text, "html.parser")

selector_films = "#mw-content-text div ul:nth-of-type(3) li i a"
films_blanchett = soup_blanchett.select(selector_films)

films_data = [
    {
        "titre": film.attrs["title"],
        "url_wikipedia": url_wikipedia + film.attrs["href"]
    }
    for film in films_blanchett
    if not (
        film.attrs.get("class") == ["new"] # Film sans page
        or film.attrs["title"] == "Galadriel" # Mauvais lien
    )
]

films = pd.DataFrame(films_data)

films

,titre,url_wikipedia
0,Les Larmes d'un homme,https://fr.wikipedia.org/wiki/Les_Larmes_d%27u...
1,Intuitions,https://fr.wikipedia.org/wiki/Intuitions
2,"Bandits (film, 2001)","https://fr.wikipedia.org/wiki/Bandits_(film,_2..."
3,Le Seigneur des anneaux : La Communauté de l'a...,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...
4,Charlotte Gray,https://fr.wikipedia.org/wiki/Charlotte_Gray
5,Terre Neuve (film),https://fr.wikipedia.org/wiki/Terre_Neuve_(film)
6,"Heaven (film, 2002)","https://fr.wikipedia.org/wiki/Heaven_(film,_2002)"
7,Le Seigneur des anneaux : Les Deux Tours,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...
8,Veronica Guerin (film),https://fr.wikipedia.org/wiki/Veronica_Guerin_...
9,Coffee and Cigarettes,https://fr.wikipedia.org/wiki/Coffee_and_Cigar...


Le sélecteur CSS que nous avons utilisé ne permettait pas d'obtenir la réponse à notre question car il ne capturait pas toutes les listes d'acteurs (organisation différente pour *Coffee and Cigarettes*, double colonne pour *Aviator*, ...). En effet, les pages Wikipedia des films ne sont pas uniformes et il n'est pas possible d'extraire la distribution de tous les films avec le même sélecteur.

Pour remédier à cela, nous proposons ici d'aller scraper la liste des acteurs sur le site [TMDB](https://www.themoviedb.org/) (*The Movie Database*) dont les pages obéissent toutes à la même organisation. Les pages Wikipedia relatives à des films contiennent toutes un lien externe vers ce site.

1. Pour chaque film, scraper la page Wikipedia pour récupérer le lien vers la page TMDB associée et déduire le lien du casting complet qui ser ajouté le dans une nouvelle colonne du dataframe `films`.

In [96]:
url = films.url_wikipedia[0]
print(url)


def get_tmdb_url(url):
    r =  requests.get(url)
    assert r.status_code == 200, "Error"

    soup = BeautifulSoup(r.text)

    selector= "a.external"
    for external_url in soup.select(selector):
        if external_url.text == "The Movie Database":
            return external_url.attrs["href"]
get_tmdb_url(url)

https://fr.wikipedia.org/wiki/Les_Larmes_d%27un_homme


'https://www.themoviedb.org/movie/29572'

2. La liste des acteurs d'un film se présente comme une liste ordonnée `ol` dans les pages TMDB. Scraper les pages de casting pour ajouter la liste des acteurs de chaque film dans une nouvelle colonne du dataframe `films`.

In [97]:
films["url_tmdb"] = films.url_wikipedia.apply(get_tmdb_url)
films

,titre,url_wikipedia,url_tmdb
0,Les Larmes d'un homme,https://fr.wikipedia.org/wiki/Les_Larmes_d%27u...,https://www.themoviedb.org/movie/29572
1,Intuitions,https://fr.wikipedia.org/wiki/Intuitions,https://www.themoviedb.org/movie/2046
2,"Bandits (film, 2001)","https://fr.wikipedia.org/wiki/Bandits_(film,_2...",https://www.themoviedb.org/movie/3172
3,Le Seigneur des anneaux : La Communauté de l'a...,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...,https://www.themoviedb.org/movie/120
4,Charlotte Gray,https://fr.wikipedia.org/wiki/Charlotte_Gray,https://www.themoviedb.org/movie/12660
5,Terre Neuve (film),https://fr.wikipedia.org/wiki/Terre_Neuve_(film),https://www.themoviedb.org/movie/6440
6,"Heaven (film, 2002)","https://fr.wikipedia.org/wiki/Heaven_(film,_2002)",https://www.themoviedb.org/movie/10575
7,Le Seigneur des anneaux : Les Deux Tours,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...,https://www.themoviedb.org/movie/121
8,Veronica Guerin (film),https://fr.wikipedia.org/wiki/Veronica_Guerin_...,https://www.themoviedb.org/movie/10629
9,Coffee and Cigarettes,https://fr.wikipedia.org/wiki/Coffee_and_Cigar...,https://www.themoviedb.org/movie/883


3. Utiliser le résultat de la question précédente pour répondre à la question initiale : avec quels acteurs Cate Blanchett a-t-elle partagé l'affiche le plus souvent au cours des années 2000 ?

In [131]:
def get_casting(url_tmdb):
    url = url_tmdb + "/cast"
    r =  requests.get(url)
    assert r.status_code == 200, "Error"

    soup = BeautifulSoup(r.text)

    selector= "section.panel:nth-of-type(1) ol.people.credits div p a"
    return [acteur.text for acteur in soup.select(selector)]



In [132]:
films["casting"] = films.url_tmdb.apply(get_casting)
films

,titre,url_wikipedia,url_tmdb,casting
0,Les Larmes d'un homme,https://fr.wikipedia.org/wiki/Les_Larmes_d%27u...,https://www.themoviedb.org/movie/29572,"[Christina Ricci, Johnny Depp, Cate Blanchett,..."
1,Intuitions,https://fr.wikipedia.org/wiki/Intuitions,https://www.themoviedb.org/movie/2046,"[Cate Blanchett, Giovanni Ribisi, Keanu Reeves..."
2,"Bandits (film, 2001)","https://fr.wikipedia.org/wiki/Bandits_(film,_2...",https://www.themoviedb.org/movie/3172,"[Bruce Willis, Billy Bob Thornton, Cate Blanch..."
3,Le Seigneur des anneaux : La Communauté de l'a...,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...,https://www.themoviedb.org/movie/120,"[Elijah Wood, Ian McKellen, Viggo Mortensen, S..."
4,Charlotte Gray,https://fr.wikipedia.org/wiki/Charlotte_Gray,https://www.themoviedb.org/movie/12660,"[Cate Blanchett, Billy Crudup, Michael Gambon,..."
5,Terre Neuve (film),https://fr.wikipedia.org/wiki/Terre_Neuve_(film),https://www.themoviedb.org/movie/6440,"[Kevin Spacey, Julianne Moore, Cate Blanchett,..."
6,"Heaven (film, 2002)","https://fr.wikipedia.org/wiki/Heaven_(film,_2002)",https://www.themoviedb.org/movie/10575,"[Cate Blanchett, Giovanni Ribisi, Remo Girone,..."
7,Le Seigneur des anneaux : Les Deux Tours,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...,https://www.themoviedb.org/movie/121,"[Elijah Wood, Ian McKellen, Viggo Mortensen, S..."
8,Veronica Guerin (film),https://fr.wikipedia.org/wiki/Veronica_Guerin_...,https://www.themoviedb.org/movie/10629,"[Cate Blanchett, Gerard McSorley, Ciarán Hinds..."
9,Coffee and Cigarettes,https://fr.wikipedia.org/wiki/Coffee_and_Cigar...,https://www.themoviedb.org/movie/883,"[Roberto Benigni, Steven Wright, Joie Lee, Cin..."


In [133]:
films.casting.explode().value_counts()

casting
Cate Blanchett         23
Peter Jackson           4
Hugo Weaving            4
Jarl Benzon             3
Sam Kelly               3
                       ..
Barry Davis             1
Anna Tzelniker          1
Danny Scheinmann        1
Claudia Lander-Duke     1
Don Fellows             1
Name: count, Length: 1245, dtype: int64